In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams

from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, AutoETS
from statsforecast.arima import arima_string

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

from src.data.loaders import CommodityLoader
from src.utils.split_series import split_series
from src.evaluation.evaluation import MetricEvaluator

import pmdarima as pm
from sktime.forecasting.exp_smoothing import ExponentialSmoothing as SKETS
from sktime.forecasting.base import ForecastingHorizon

/home/hiraeth/anaconda3/envs/smps/lib/python3.10/site-packages/fs/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore


In [14]:
def get_arima_orders(sf_arima):
    info = getattr(sf_arima, "model_", None)
    if not isinstance(info, dict) or "arma" not in info:
        raise ValueError("Não foi possível encontrar 'arma' em sf_arima.model_")
    p, d, q, P, D, Q, m = info["arma"]
    order = (int(p), int(d), int(q))
    seasonal_order = None if m <= 1 or (P, D, Q) == (0, 0, 0) else (int(P), int(D), int(Q), int(m))
    return order, seasonal_order

def get_ets_spec(sf_ets, default_sp):
    info = getattr(sf_ets, "model_", None)
    code = info.get("model", "ANN")
    trend_code = code[1] if len(code) > 1 else "N"
    seas_code  = code[2] if len(code) > 2 else "N"
    trend = {"A": "add", "M": "mul", "N": None}.get(trend_code, None)
    seasonal = {"A": "add", "M": "mul", "N": None}.get(seas_code, None)
    phi = info.get("phi", None)
    damped = (phi is not None) and (phi != 1)
    sp = int(info.get("m", info.get("season_length", default_sp)))
    return {"trend": trend, "seasonal": seasonal, "damped_trend": damped, "sp": sp}

In [15]:
# ---------------------------
# Rollout multi-série
# ---------------------------
def one_step_multiseries(full_train, test, models_trained, season_length=12):
    first_row = models_trained[0]
    col_map = {type(first_row[j]).__name__: j for j in range(len(first_row))}
    j_arima = col_map["AutoARIMA"]
    j_ets   = col_map["AutoETS"]

    uids = full_train["unique_id"].unique().tolist()
    all_preds = []

    for i, uid in enumerate(uids):
        y_tr = full_train[full_train["unique_id"] == uid].set_index("ds")["y"].astype(float)
        y_te = test[test["unique_id"] == uid].set_index("ds")["y"].astype(float)
        if y_te.empty: 
            continue

        sf_arima = models_trained[i, j_arima]
        sf_ets   = models_trained[i, j_ets]

        # ARIMA (pmdarima)
        order, seasonal_order = get_arima_orders(sf_arima)
        if seasonal_order is None:
            arima = pm.ARIMA(order=order, suppress_warnings=True)
        else:
            arima = pm.ARIMA(order=order, seasonal_order=seasonal_order, suppress_warnings=True)
        arima.fit(y_tr.to_timestamp())  # pmdarima usa DatetimeIndex

        # ETS (sktime)
        ets_spec = get_ets_spec(sf_ets, default_sp=season_length)
        ets = SKETS(
            trend=ets_spec["trend"],
            damped_trend=ets_spec["damped_trend"],
            seasonal=ets_spec["seasonal"],
            sp=ets_spec["sp"],
            initialization_method="estimated",
        )
        ets.fit(y_tr)

        preds = []
        for ts, y_true in y_te.items():
            # ARIMA
            yhat_arima = arima.predict(1)[0]
            arima.update(y_true)

            # ETS
            fh = ForecastingHorizon(pd.PeriodIndex([ts], freq="M"), is_relative=False)
            yhat_ets = ets.predict(fh).iloc[0]
            ets.update(
                pd.Series([y_true], index=pd.PeriodIndex([ts], freq="M")),
                update_params=False
            )

            preds.append({
                "unique_id": uid,
                "ds": ts,
                "yhat_arima": float(yhat_arima),
                "yhat_ets": float(yhat_ets),
            })

        all_preds.append(pd.DataFrame(preds))

    return pd.concat(all_preds, ignore_index=True)


In [2]:
df_brl = CommodityLoader.load_all_commodities(
    currency='BRL',
    preprocessing=True,
    monthly_aggregation='mean',
    limit_date=None
)

ദ്ദി・ᴗ・)✧ acucar_santos carregado com sucesso
ദ്ദി・ᴗ・)✧ acucar_sp carregado com sucesso
ദ്ദി・ᴗ・)✧ algodao carregado com sucesso
ദ്ദി・ᴗ・)✧ arroz carregado com sucesso
ദ്ദി・ᴗ・)✧ cafe_arabica carregado com sucesso
ദ്ദി・ᴗ・)✧ cafe_robusta carregado com sucesso
ദ്ദി・ᴗ・)✧ milho carregado com sucesso
ദ്ദി・ᴗ・)✧ soja_parana carregado com sucesso
ദ്ദി・ᴗ・)✧ soja_paranagua carregado com sucesso
ദ്ദി・ᴗ・)✧ trigo_parana carregado com sucesso
ദ്ദി・ᴗ・)✧ trigo_rs carregado com sucesso


In [3]:
train, val, test = split_series(df_brl)
full_train = pd.concat([train, val])

Splitted data into TRAIN with size 1605, VALIDATION with size 406 and TEST with size 868


In [4]:
season_length = 12

models = [
    AutoARIMA(season_length=season_length),
    AutoETS(model="AZN")
]

sf = StatsForecast(models=models, freq='MS')
sf_models = sf.fit(full_train)

In [6]:
models_trained = sf_models.fitted_
print(models_trained)

[[AutoARIMA AutoETS]
 [AutoARIMA AutoETS]
 [AutoARIMA AutoETS]
 [AutoARIMA AutoETS]
 [AutoARIMA AutoETS]
 [AutoARIMA AutoETS]
 [AutoARIMA AutoETS]
 [AutoARIMA AutoETS]
 [AutoARIMA AutoETS]
 [AutoARIMA AutoETS]
 [AutoARIMA AutoETS]]


In [9]:
first_row = models_trained[0]
print(first_row)

col_map = {type(first_row[j]).__name__: j for j in range(len(first_row))}
print(col_map)

j_arima = col_map["AutoARIMA"]
j_ets   = col_map["AutoETS"]
print(j_arima, j_ets)

[AutoARIMA AutoETS]
{'AutoARIMA': 0, 'AutoETS': 1}
0 1


In [20]:
uids = full_train["unique_id"].unique().tolist()
all_preds = []

print(uids)

for i, uid in enumerate(uids):
    print(i, uid)
    y_tr = full_train[full_train["unique_id"] == uid].set_index("ds")["y"].astype(float)
    y_te = test[test["unique_id"] == uid].set_index("ds")["y"].astype(float)
    print(type(y_tr))
    print(type(y_te))

    sf_arima = models_trained[i, j_arima]
    sf_ets   = models_trained[i, j_ets]
    print(sf_arima, type(sf_arima))
    print(sf_ets, type(sf_ets))

    info = getattr(sf_arima, "model_", None)
    print(info)
    p, d, q, P, D, Q, m = info["arma"]
    order = (int(p), int(d), int(q))
    seasonal_order = None if m <= 1 or (P, D, Q) == (0, 0, 0) else (int(P), int(D), int(Q), int(m))
    print(order, seasonal_order)

    order, seasonal_order = get_arima_orders(sf_arima)
    print(order, seasonal_order)

    ets_spec = get_ets_spec(sf_ets, default_sp=season_length)
    print(ets_spec)

['ACUCAR_SANTOS', 'ACUCAR_SP', 'ALGODAO', 'ARROZ', 'CAFE_ARABICA', 'CAFE_ROBUSTA', 'MILHO', 'SOJA_PARANA', 'SOJA_PARANAGUA', 'TRIGO_PARANA', 'TRIGO_RS']
0 ACUCAR_SANTOS
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
AutoARIMA <class 'statsforecast.models.AutoARIMA'>
AutoETS <class 'statsforecast.models.AutoETS'>
{'coef': {'drift': 2.289644628099174}, 'sigma2': 34.32224803783469, 'var_coef': array([[0.02272727]]), 'mask': array([ True]), 'loglik': -139.71491366568836, 'aic': 283.4298273313767, 'arma': (0, 0, 0, 0, 1, 1, 0), 'residuals': array([  0.07113896,   5.91783012,  -4.0193921 ,  -5.08250826,
         5.23535537,  -4.34814463,  -3.65355767,   5.23903032,
        -5.34916844,   9.88987918,  -2.01940653,  -7.40414463,
         9.85880274,   3.70696356,  -0.7373741 ,  -2.42747072,
        -2.38012082,  -8.32059701,   1.80368871,   8.5393597 ,
        -0.55198229,   4.63328394,  -0.41264463,   0.86328394,
        -5.90202558, -10.52185014,   2.57305872,  -7.01